In [2]:
import os
import sys
import urllib3
from urllib.parse import urlparse
import pandas as pd
import itertools
import shutil

from urllib3.util import Retry

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

classes = ["cat", "fish"]
set_types = ["train", "test", "val"]

def download_image(url, klass, data_type):
    basename = os.path.basename(urlparse(url).path)
    filename = "{}/{}/{}".format(data_type, klass, basename)
    if not os.path.exists(filename):
        try: 
            http = urllib3.PoolManager(retries=Retry(connect=1, read=1, redirect=2))
            with http.request("GET", url, preload_content=False) as resp, open(
                filename, "wb"
            ) as out_file:
                if resp.status == 200:
                    shutil.copyfileobj(resp, out_file)
                else:
                    print("Error downloading {}".format(url))
            resp.release_conn()
        except:
            print("Error downloading {}".format(url))


if __name__ == "__main__":
    if not os.path.exists("images.csv"):
        print("Error: can't find images.csv!")
        sys.exit(0)

    # get args and create output directory
    imagesDF = pd.read_csv("images.csv")

    for set_type, klass in list(itertools.product(set_types, classes)):
        path = "./{}/{}".format(set_type, klass)
        if not os.path.exists(path):
            print("Creating directory {}".format(path))
            os.makedirs(path)

    print("Downloading {} images".format(len(imagesDF)))

    result = [
        download_image(url, klass, data_type)
        for url, klass, data_type in zip(
            imagesDF["url"], imagesDF["class"], imagesDF["type"]
        )
    ]
    sys.exit(0)

Creating directory ./train/cat
Creating directory ./train/fish
Creating directory ./test/cat
Creating directory ./test/fish
Creating directory ./val/cat
Creating directory ./val/fish
Error downloading http://farm1.static.flickr.com/22/31377472_8b8b15d01b.jpg
Error downloading http://farm1.static.flickr.com/1/1004525_cba96ba3c3.jpg
Error downloading http://farm1.static.flickr.com/33/65048097_e5264bf855.jpg
Error downloading http://farm3.static.flickr.com/2354/2102976081_61c8614be8.jpg
Error downloading http://farm2.static.flickr.com/1390/709949156_5e4ac3f499.jpg
Error downloading http://farm1.static.flickr.com/47/126084132_93e5aeade7.jpg
Error downloading http://farm2.static.flickr.com/1221/1011749126_44a195db4c.jpg
Error downloading http://farm1.static.flickr.com/173/416994740_6ada308baa.jpg
Error downloading http://farm1.static.flickr.com/3/4193130_a058cdb81f.jpg
Error downloading http://farm1.static.flickr.com/1/1004528_a111209743.jpg
Error downloading http://farm1.static.flickr.com/

SystemExit: 0

/home/science/projects/nlp_university/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
